# Index calculus algorithm

We will perform the Index calculus algorithm to compute $\log_{a} b_i$ ($i = 1, 2$) in $\mathbf{Z}_{p}^*$, where $a = 5$, $b_1 = 4389733$, $b_2 = 1234567$, and $p = 9330887$ is a prime number.

In [1]:
from algorithms.euclidean import gcd
from algorithms.factorization import factorizeByBase

p = 9330887
a = 5
b1 = 4389733
b2 = 1234567

We will use a base consisting of $-1$ and all primes less than $50$.

In [2]:
base = [-1] + prime_range(50)

Let us now try to find the logarithms table. We construct a matrix by trying random powers of $a$ and factoring them with the numbers in our base. We stop when the matrix has full rank.

In [3]:
set_random_seed(0)

v = []
s = set()
r, l = 0, len(base)
M = Matrix(nrows=0, ncols=l)
while r < l:
    i = randint(1, p-1)
    if i in s:
        continue
    s.add(i)
    x = pow(a, i, p)
    f = factorizeByBase(Integer(x), base, p)
    if not f:
        continue
    MM = Matrix(list(M) + [f])
    rr = MM.rank()
    if rr > r:
        M = MM
        r = rr
        v.append(i)
print(M)
print(v)

[1 0 1 0 1 0 1 1 0 0 0 0 1 0 1 0]
[1 1 4 0 2 0 0 0 1 1 0 0 0 0 0 0]
[1 1 5 2 1 0 0 1 0 0 0 0 0 0 0 0]
[1 6 0 0 0 1 1 0 0 0 0 0 0 1 0 0]
[0 3 1 0 0 1 0 0 0 0 1 1 0 0 0 0]
[0 2 0 2 0 0 1 0 0 0 1 0 0 0 0 1]
[1 9 3 2 1 0 0 0 0 0 0 0 0 0 0 0]
[0 1 1 2 0 0 0 0 0 0 1 1 0 1 0 0]
[0 3 4 1 0 0 1 0 1 0 0 0 0 0 0 0]
[0 1 0 0 3 0 0 0 1 0 0 0 0 0 0 1]
[0 3 3 0 0 0 0 0 2 0 0 0 0 0 1 0]
[1 2 0 0 0 2 0 0 0 0 0 1 0 1 0 0]
[1 6 3 0 1 0 0 2 0 0 0 0 0 0 0 0]
[1 0 0 1 2 0 0 0 0 1 2 0 0 0 0 0]
[1 6 0 0 0 1 1 0 0 0 1 1 0 0 0 0]
[0 3 1 1 1 0 1 0 1 0 0 0 1 0 0 0]
[4746917, 4747026, 6889022, 3908343, 1066760, 6494367, 1934471, 1567977, 7864629, 1638582, 2526531, 6829987, 6466117, 3166568, 4256144, 5639062]


We now solve the system of equations we have obtained. The solution represents our table of logarithms and can be used to find logarithms of multiple numbers.

In [4]:
t = M^-1 * Matrix(zip(v)) % (p-1)
t

[4665443]
[6670912]
[2030334]
[      1]
[5786904]
[1078197]
[8534197]
[7606749]
[8519903]
[2519168]
[6200403]
[9068634]
[7409417]
[5590350]
[6037417]
[6410599]

Let us verify that the table is correct.

In [5]:
[pow(a, i, p) for i, in t]

[9330886, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]

We can now find a power of $b_i$ that can be factorized with our base. We will use the following function.

In [6]:
def findPower(b, p, t, base):
    while True:
        i = randint(1, p-1)
        x = pow(b, i, p)
        f = factorizeByBase(Integer(x), base, p)
        if not f:
            continue
        return (i, f)

Let us find such a power of $b_1$ and its factorization.

In [7]:
i1, f1 = findPower(b1, p, t, base)
i1, f1

(7675068, [0, 1, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0])

We can now use the logarithm table to obtain a congruence in $\log_a b_1$. However, there may be multiple solutions, and we need to check which one is our answer. The number of solutions is given by the greatest common divisor of $p-1$ and the obtained exponent.

In [8]:
g1 = gcd(i1, p-1)
g1

2

We can thus obtain a solution modulo $p-1$ divided by this GCD.

In [9]:
m1 = ((p-1)/g1)
(x1,), = Matrix([f1])*t / i1 % m1
x1

1087862

Let us now put this into a function which will also verify the potential solutions.

In [10]:
def checkSolutions(a, b, p, t, i, f):
    g = gcd(i, p-1)
    m = (p-1)/g
    (x,), = Matrix([f])*t / i % m
    for j in range(g):
        y = pow(a, x, p)
        print("%d^%d mod %d = %d" % (a, x, p, y))
        if y == b:
            return x
        x += m

checkSolutions(a, b1, p, t, i1, f1)

5^1087862 mod 9330887 = 4941154
5^5753305 mod 9330887 = 4389733


5753305

We have thus computed $\log_a b_1$. Let us now compute $\log_a b_2$.

In [11]:
i2, f2 = findPower(b2, p, t, base)
i2, f2

(1643897, [1, 7, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [12]:
checkSolutions(a, b2, p, t, i2, f2)

5^7273774 mod 9330887 = 1234567


7273774

We will now use the function `logarithmTable` to compute tables of logarithms, which will then be used to compute some more discrete logarithms with the function `indexCalculus`. We will measure the evaluation times.

In [13]:
from algorithms.discreteLogarithm import logarithmTable, indexCalculus

aa = 47
bb = 191

In [14]:
pp = 100000000003
table = %time logarithmTable(aa, pp, base, trace = True)
print(table)
%time indexCalculus(aa, bb, pp, base, table, trace = True)

found factorization 47^80518442549 = 5^3 * 11^1 * 13^2 * 17^1 * 29^1 * 37^1 (mod 100000000003)
found factorization 47^84861465387 = -1^1 * 2^1 * 3^1 * 5^7 * 7^1 * 43^1 (mod 100000000003)
found factorization 47^90432226991 = 2^1 * 3^1 * 5^4 * 7^4 * 13^1 * 17^1 * 19^1 (mod 100000000003)
found factorization 47^76313756410 = -1^1 * 2^6 * 3^1 * 5^2 * 7^1 * 19^2 * 23^1 * 43^1 (mod 100000000003)
found factorization 47^73886231161 = 2^4 * 3^1 * 5^2 * 7^1 * 11^1 * 41^1 (mod 100000000003)
found factorization 47^20455038019 = -1^1 * 2^3 * 7^3 * 11^1 * 19^1 * 23^1 * 43^1 (mod 100000000003)
found factorization 47^13620877439 = -1^1 * 2^1 * 3^2 * 7^1 * 13^1 * 17^1 * 23^1 * 31^2 * 37^1 (mod 100000000003)
found factorization 47^80102434804 = -1^1 * 2^4 * 3^1 * 7^2 * 11^1 * 23^1 * 37^2 * 41^1 (mod 100000000003)
found factorization 47^27198933959 = -1^1 * 2^1 * 5^1 * 13^1 * 23^2 * 31^2 * 41^1 (mod 100000000003)
found factorization 47^82442132303 = 2^8 * 3^1 * 7^1 * 29^2 * 41^1 (mod 100000000003)
found f

6935101882

In [15]:
pp = 10000000000000000051
base = [-1] + prime_range(5000)
table = %time logarithmTable(aa, pp, base, trace = True)
%time indexCalculus(aa, bb, pp, base, table, trace = True)

found factorization 47^953676781484914302 = 2^1 * 29^1 * 1627^1 * 2357^1 * 2879^1 * 3571^1 * 3943^1 (mod 10000000000000000051)
found factorization 47^6287360725992047568 = -1^1 * 2^3 * 5^1 * 11^2 * 19^1 * 827^1 * 1879^1 * 3793^1 * 4651^1 (mod 10000000000000000051)
found factorization 47^1962320992248740799 = 3^1 * 5^4 * 7^1 * 11^1 * 31^1 * 211^1 * 607^1 * 797^1 * 3643^1 (mod 10000000000000000051)
found factorization 47^1819364495386436834 = -1^1 * 2^1 * 3^1 * 5^3 * 23^1 * 167^1 * 173^1 * 1487^1 * 1531^1 * 3733^1 (mod 10000000000000000051)
found factorization 47^8263301872670395073 = -1^1 * 2^1 * 3^2 * 11^1 * 31^1 * 431^1 * 727^1 * 751^1 * 887^1 * 2677^1 (mod 10000000000000000051)
found factorization 47^3113345274525834062 = 2^6 * 5^1 * 31^1 * 127^1 * 541^1 * 641^1 * 2593^1 * 3623^1 (mod 10000000000000000051)
found factorization 47^8533724016359759095 = -1^1 * 2^1 * 7^1 * 23^1 * 349^1 * 389^1 * 1471^1 * 3037^1 * 3769^1 (mod 10000000000000000051)
found factorization 47^420261329710197431

found factorization 47^9559994506038927750 = 3^2 * 11^1 * 131^1 * 307^1 * 331^1 * 991^1 * 2281^1 * 2539^1 (mod 10000000000000000051)
found factorization 47^6994716761782100659 = -1^1 * 7^4 * 11^1 * 13^1 * 17^1 * 41^1 * 107^1 * 293^1 * 439^1 * 2293^1 (mod 10000000000000000051)
found factorization 47^6149657592562131115 = -1^1 * 29^1 * 151^1 * 523^1 * 727^1 * 1013^1 * 1129^1 * 4451^1 (mod 10000000000000000051)
found factorization 47^6568112653502021422 = 3^4 * 107^1 * 113^1 * 593^1 * 1699^1 * 2011^1 * 3203^1 (mod 10000000000000000051)
found factorization 47^9431569325583918761 = 3^2 * 29^1 * 67^1 * 127^1 * 421^1 * 1439^1 * 2143^1 * 2659^1 (mod 10000000000000000051)
found factorization 47^6925674726689004428 = -1^1 * 2^2 * 3^1 * 11^1 * 137^1 * 227^1 * 311^1 * 383^1 * 751^1 * 2309^1 (mod 10000000000000000051)
found factorization 47^3340518389169749596 = -1^1 * 3^1 * 11^1 * 97^1 * 439^2 * 719^1 * 907^1 * 1399^1 (mod 10000000000000000051)
found factorization 47^7430889324928258848 = 2^2 * 3^

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/janos/sage/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/janos/sage/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/home/janos/sage/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/home/janos/sage/local/lib/python2.7/inspect.py", line 1053, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/home/janos/sage/local/lib/python2.7/inspect.py", line 1013, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/home/janos/sage/local/lib/python2.7/inspect.py", line 453, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File

IndexError: string index out of range